In [ ]:
%pip install firebase-admin

In [4]:
# import firebase_admin
# from firebase_admin import credentials, firestore

# try:
#     # Initialize Firebase
#     cred = credentials.Certificate('/home/jericho/Documents/GitHub/deeplearning/projects/mental-health-bot/Backend/wellcarebot-71cca-firebase-adminsdk-v1v74-7337868583.json')
#     firebase_admin.initialize_app(cred)
#     db = firestore.client()
#     print("Firebase initialized successfully.")
# except Exception as e:
#     print(f"Error initializing Firebase: {e}")


Firebase initialized successfully.


ADD  AND UPDATE THERAPIST

In [8]:
def add_or_update_therapist(therapist_id, therapist_data):
    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_ref.set(therapist_data, merge=True)
    return "Therapist added/updated successfully."

# Example usage
therapist_data = {
    'name': 'Dr. Mary Jane Doe',
    'specialization': 'Psychologist',
    'location': 'Kampala',
    'availability': ['2024-08-10T10:00:00', '2024-08-11T14:00:00'],
    'contact_info': 'jane.doe@example.com'
}

print(add_or_update_therapist('therapist_1234', therapist_data))


Therapist added/updated successfully.


create a booking

In [10]:
def create_booking(user_id, therapist_id, appointment_date):
    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_doc = therapist_ref.get()
    therapist_data = therapist_doc.to_dict()

    if not therapist_data or appointment_date not in therapist_data.get('availability', []):
        return "Slot not available"

    booking_ref, _ = db.collection('bookings').add({
        'user_id': user_id,
        'therapist_id': therapist_id,
        'appointment_date': appointment_date,
        'status': 'confirmed',
        'notes': ''
    })

    # Update therapist availability
    therapist_ref.update({
        'availability': firestore.ArrayRemove([appointment_date])
    })

    return f"Booking created successfully. Booking ID: {booking_ref.id}"

# Example usage
print(create_booking('Peter', 'therapist_123', '2024-08-10T10:00:00'))


Slot not available


retrive therapist availabilty

In [11]:
def get_therapist_availability(therapist_id):
    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_doc = therapist_ref.get()
    if therapist_doc.exists:
        return therapist_doc.to_dict().get('availability', [])
    else:
        return "Therapist not found"

# Example usage
print(get_therapist_availability('therapist_123'))


['2024-08-11T14:00:00']


list user bookings

In [13]:
def list_user_bookings(user_id):
    bookings_ref = db.collection('bookings')
    query = bookings_ref.where('user_id', '==', user_id).stream()
    bookings = [booking.to_dict() for booking in query]
    return bookings

# Example usage
print(list_user_bookings('Peter'))


[{'user_id': 'Peter', 'therapist_id': 'therapist_123', 'status': 'confirmed', 'appointment_date': '2024-08-10T10:00:00', 'notes': ''}]


In [ ]:
def create_booking(user_id, therapist_id, appointment_date):
    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_doc = therapist_ref.get()
    therapist_data = therapist_doc.to_dict()

    if not therapist_data or appointment_date not in therapist_data.get('availability', []):
        return "Slot not available"

    booking_ref, _ = db.collection('bookings').add({
        'user_id': user_id,
        'therapist_id': therapist_id,
        'appointment_date': appointment_date,
        'status': 'confirmed',
        'notes': ''
    })

    # Update therapist availability
    therapist_ref.update({
        'availability': firestore.ArrayRemove([appointment_date])
    })

    return f"Booking created successfully. Booking ID: {booking_ref.id}"

# Example usage
print(create_booking('Peter', 'therapist_123', '2024-08-10T10:00:00'))


Slot not available


In [14]:
def get_user_id_by_email(email):
    users_ref = db.collection('users')
    query = users_ref.where('email', '==', email).limit(1)
    results = query.stream()

    for doc in results:
        return doc.id

    return None

# Example usage
user_id = get_user_id_by_email('katendejericho5@gmail.com')
print(user_id)


fnHeUqUclxM9Ybbx5mEf0Tcd7XU2


In [24]:
def get_user_by_email(email):
    users_ref = db.collection('users')
    query = users_ref.where('email', '==', email).limit(1)
    results = query.stream()

    for doc in results:
        print(f"User found: {doc.id}")
        return doc.id, doc.to_dict()

    print("User not found")
    return None, None

def create_booking(email, therapist_id, appointment_date):
    user_id, user_data = get_user_by_email(email)
    if not user_id:
        return "User not found"

    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_doc = therapist_ref.get()
    therapist_data = therapist_doc.to_dict()

    # Print the therapist's availability for debugging
    print(f"Therapist availability: {therapist_data.get('availability', [])}")

    if not therapist_data or appointment_date not in therapist_data.get('availability', []):
        return "Slot not available"

    booking_data = {
        'user_id': user_id,
        'user_name': user_data['fullName'],
        'user_email': user_data['email'],
        'user_phone': user_data['phoneNumber'],
        'user_profile_picture_url': user_data['profilePictureURL'],
        'therapist_id': therapist_id,
        'appointment_date': appointment_date,
        'status': 'confirmed',
        'notes': ''
    }

    booking_ref = db.collection('bookings').add(booking_data)[1]  # Extracting the DocumentReference from the tuple

    # Update therapist availability
    therapist_ref.update({
        'availability': firestore.ArrayRemove([appointment_date])
    })

    print(f"Booking created successfully. Booking ID: {booking_ref.id}")
    return f"Booking created successfully. Booking ID: {booking_ref.id}"

# Example usage with an available date
print(create_booking('katendejericho5@gmail.com', 'therapist_1234', '2024-08-11T14:00:00'))

User found: fnHeUqUclxM9Ybbx5mEf0Tcd7XU2
Therapist availability: ['2024-08-10T10:00:00', '2024-08-11T14:00:00']
Booking created successfully. Booking ID: xy4i3aVIJ6QKK3VZlV5T
Booking created successfully. Booking ID: xy4i3aVIJ6QKK3VZlV5T


changed time foramt

In [31]:
from datetime import datetime


def get_user_by_email(email):
    users_ref = db.collection('users')
    query = users_ref.where('email', '==', email).limit(1)
    results = query.stream()

    for doc in results:
        print(f"User found: {doc.id}")
        return doc.id, doc.to_dict()

    print("User not found")
    return None, None

def create_booking(email, therapist_id, appointment_date):
    user_id, user_data = get_user_by_email(email)
    if not user_id:
        return "User not found"

    therapist_ref = db.collection('therapists').document(therapist_id)
    therapist_doc = therapist_ref.get()
    therapist_data = therapist_doc.to_dict()

    # Print the therapist's availability for debugging
    print(f"Therapist availability: {therapist_data.get('availability', [])}")

    if not therapist_data:
        return "Therapist not found"
    
    # Check if the appointment_date is in the correct format
    try:
        appointment_datetime = datetime.fromisoformat(appointment_date)
        print(f"Parsed appointment_date: {appointment_datetime}")
    except ValueError:
        return "Invalid date format. Please use YYYY-MM-DDTHH:MM:SS"

    year = appointment_datetime.year
    month = appointment_datetime.month
    day = appointment_datetime.day
    time = appointment_datetime.time().isoformat()

    if appointment_date not in therapist_data.get('availability', []):
        return "Slot not available"

    booking_data = {
        'user_id': user_id,
        'user_name': user_data['fullName'],
        'user_email': user_data['email'],
        'user_phone': user_data['phoneNumber'],
        'user_profile_picture_url': user_data['profilePictureURL'],
        'therapist_id': therapist_id,
        'appointment_date': appointment_date,
        'year': year,
        'month': month,
        'day': day,
        'time': time,
        'status': 'confirmed',
        'notes': ''
    }

    # Add booking to the 'bookings' collection
    booking_ref = db.collection('bookings').add(booking_data)[1]  # Extracting the DocumentReference from the tuple

    # Update therapist availability
    therapist_ref.update({
        'availability': firestore.ArrayRemove([appointment_date])
    })

    print(f"Booking created successfully. Booking ID: {booking_ref.id}")
    return f"Booking created successfully. Booking ID: {booking_ref.id}"

# Example usage with an available date
print(create_booking('jerichokatende5@gmail.com', 'therapist_1234', '2024-08-10T10:00:00'))

User found: HEklKe4yVMgPcVmOONUkehKJfxh2
Therapist availability: ['2024-08-10T10:00:00']
Parsed appointment_date: 2024-08-10 10:00:00
Booking created successfully. Booking ID: 9DPvYAXQqafWw8zlJnUn
Booking created successfully. Booking ID: 9DPvYAXQqafWw8zlJnUn
